In [2]:
import ee
import numpy as np
import pandas as pd

In [2]:
ee.Authenticate()



Successfully saved authorization token.


In [4]:
ee.Initialize(project='ee-apr160')
print(ee.String('Hello from the Earth Engine servers!').getInfo())


Hello from the Earth Engine servers!


In [64]:
# compute NDVI from NIR and red band in sentinel -2 image
# For other satellite image, please change the band information accordingly

def getNDVI(image):
    
    # Normalized difference vegetation index (NDVI)
    ndvi = image.normalizedDifference(['B8','B4']).rename("NDVI")
    image = image.addBands(ndvi)

    return(image)

def getNDVI2(image):
    
    # Normalized difference vegetation index (NDVI)
    ndvi = image.normalizedDifference(['NDVI'])
    image = image.addBands(ndvi)

    return(image)


# compute EVI from NIR and red band in sentinel -2 image
# For other satellite image, please change the band information accordingly

def getEVI(image):
    # Compute the EVI using an expression.
    EVI = image.expression(
        '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
            'NIR': image.select('B8').divide(10000),
            'RED': image.select('B4').divide(10000),
            'BLUE': image.select('B2').divide(10000)
        }).rename("EVI")

    image = image.addBands(EVI)

    return(image)

# manage the date formating as per your requirements
# Mine is in format of YYYYMMdd
def addDate(image):
    img_date = ee.Date(image.date())
    img_date = ee.Number.parse(img_date.format('YYYYMMdd'))
    return image.addBands(ee.Image(img_date).rename('date').toInt())



In [69]:
Sentinel_data = ee.ImageCollection('MODIS/MOD09GA_006_NDVI') \
    .filterDate("2021-09-20","2023-12-31") \
    .map(addDate)

In [66]:
Sentinel_data = ee.ImageCollection('COPERNICUS/S2_HARMONIZED') \
    .filterDate("2021-09-20","2023-12-31") \
    .map(getNDVI).map(getEVI).map(addDate)

In [73]:
df = pd.read_csv('2022.csv',encoding = "ISO-8859-1")
df = df[['Latitud','Longitud']].drop_duplicates().reset_index(drop=True)
df.to_csv('points.csv')

In [62]:
features=[]
for index, row in df.iterrows():
#     print(dict(row))
#     construct the geometry from dataframe
    poi_geometry = ee.Geometry.Point([row['Longitud'], row['Latitud']])
#     print(poi_geometry)
#     construct the attributes (properties) for each point 
    poi_properties = dict(row)
#     construct feature combining geometry and properties
    poi_feature = ee.Feature(poi_geometry, poi_properties)
#     print(poi_feature)
    features.append(poi_feature)

# final Feature collection assembly



In [74]:
def save_coordinate(row):
    features = []
    poi_geometry = ee.Geometry.Point([row['Longitud'], row['Latitud']])
    poi_properties = dict(row)
    poi_feature = ee.Feature(poi_geometry, poi_properties)
    features.append(poi_feature)
    ee_fc = ee.FeatureCollection(features) 


    def rasterExtraction(image):
        feature = image.sampleRegions(
            collection = ee_fc, # feature collection here
            scale = 10 # Cell size of raster
        )
        return feature
    
    results = Sentinel_data.filterBounds(ee_fc).select('NDVI').map(addDate).map(rasterExtraction).flatten()
    sample_result = results.first().getInfo()

    # You can modify this for better optimization
    column_df = list(df.columns)
    column_df.extend(['NDVI', 'date'])
    
    nested_list = results.reduceColumns(ee.Reducer.toList(len(column_df)), column_df).values().get(0)
    data = nested_list.getInfo()
    #print(len(data))
    df_final = pd.DataFrame(data,columns=column_df)
    df_final.to_csv('coordinates-modis/ndvi_' + str(row['Longitud']) + '_' + str(row['Latitud']) + '.csv')
    

for index, row in df.iterrows():
    save_coordinate(row)
    print('.',end='')


........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [49]:
ee_fc = ee.FeatureCollection(features) 


def rasterExtraction(image):
    feature = image.sampleRegions(
        collection = ee_fc, # feature collection here
        scale = 10 # Cell size of raster
    )
    return feature

#ee_fc.getInfo()

In [51]:
results = Sentinel_data.filterBounds(ee_fc).select('NDVI').map(addDate).map(rasterExtraction).flatten()
sample_result = results.first().getInfo()
sample_result

{'type': 'Feature',
 'geometry': None,
 'id': '2023_01_01_0_0',
 'properties': {'Latitud': 20.85595,
  'Longitud': -90.25643,
  'NDVI': 0.6215344667434692,
  'date': 20230101}}

In [52]:
# extract the properties column from feature collection
# column order may not be as our sample data order
columns = list(sample_result['properties'].keys())
print(columns)


# Order data column as per sample data
# You can modify this for better optimization
column_df = list(df.columns)
column_df.extend(['NDVI', 'date'])
print(column_df)
nested_list = results.reduceColumns(ee.Reducer.toList(len(column_df)), column_df).values().get(0)
data = nested_list.getInfo()
data
# dont forget we need to call the callback method "getInfo" to retrieve the data
#df = pd.DataFrame(data, columns=column_df)
# we obtain the data frame as per our demand
#df.to_csv('out.csv')

['Latitud', 'Longitud', 'NDVI', 'date']
['Latitud', 'Longitud', 'NDVI', 'date']


[[20.85595, -90.25643, 0.6215344667434692, 20230101],
 [20.85498, -90.26168, 0.6215344667434692, 20230101],
 [20.81349, -90.19725, 0.7544791102409363, 20230101],
 [20.80072, -90.19945, 0.5672207474708557, 20230101],
 [20.83347, -90.21466, 0.7350199818611145, 20230101],
 [20.83276, -90.1928, 0.7666999101638794, 20230101],
 [20.76601, -90.2103, 0.6093350052833557, 20230101],
 [20.78672, -90.19702, 0.6272469162940979, 20230101],
 [21.15812, -89.50899, 0.40847569704055786, 20230101],
 [21.12299, -89.49265, 0.5026738047599792, 20230101],
 [21.12456, -89.48798, 0.5026738047599792, 20230101],
 [21.11856, -89.4963, 0.48557692766189575, 20230101],
 [21.11791, -89.51223, 0.4960000216960907, 20230101],
 [21.11377, -89.51887, 0.5093901753425598, 20230101],
 [21.12809, -89.4926, 0.5424151420593262, 20230101],
 [21.1299, -89.50571, 0.5097236633300781, 20230101],
 [21.31297, -88.67054, 0.2907109260559082, 20230101],
 [21.4095, -88.61501, 0.22465068101882935, 20230101],
 [21.38496, -88.61884, 0.218913

In [54]:
len(data)

100182

In [53]:
df_final = pd.DataFrame(data,columns=column_df)
df_final.to_csv('ndvi-2023.csv')

In [19]:
url_csv = results.getDownloadURL('csv')
# click the link below, this will download CSV directly to your local device
# You can use this url and download with python request module, I will leave that to you
url_csv


'https://earthengine.googleapis.com/v1/projects/ee-apr160/tables/4b86786de611f6e77e57f9cf022a78c0-13d2ee22535a07de01ff596228567065:getFeatures'